**Use GPU: Runtime -> Change runtime type -> GPU (Hardware Accelerator)**

Setup

In [1]:
!cat ~/.keras/keras.json

{
    "epsilon": 1e-07, 
    "floatx": "float32", 
    "image_data_format": "channels_last", 
    "backend": "tensorflow"
}

In [2]:
import keras
print(keras.__version__)

2.4.3


Neural Style

In [3]:
# import the necessary packages
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras import backend as K
from scipy.optimize import fmin_l_bfgs_b
import numpy as np
import cv2
import os

class NeuralStyle:
    def __init__(self, settings):
        # store the settings dictionary
        self.S = settings

        # grab the dimensions of the input image
        (w, h) = load_img(self.S["input_path"]).size
        self.dims = (h, w)

        # load content image and style images, forcing the dimensions
        # of our input image
        self.content = self.preprocess(settings["input_path"])
        self.style = self.preprocess(settings["style_path"])
        self.content = K.variable(self.content)
        self.style = K.variable(self.style)

        # allocate memory of our output image, then combine the
        # content, style, and output into a single tensor so they can
        # be fed through the network
        self.output = K.placeholder((1, self.dims[0], self.dims[1], 3))
        self.input = K.concatenate([self.content, self.style, self.output], axis=0)

        # load our model from disk
        print("[INFO] loading network...")
        self.model = self.S["net"](weights="imagenet", include_top=False, input_tensor=self.input)

        # build a dictionary that maps the *name* of each layer
        # inside the network to the actual layer *output*
        layerMap = {l.name: l.output for l in self.model.layers}

        # extract features from the content layer, then extract the
        # activations from the style image (index 0) and the output
        # image (index 2) -- these will serve as our style features
        # and output features from the *content* layer
        contentFeatures = layerMap[self.S["content_layer"]]
        styleFeatures = contentFeatures[0, :, :, :]
        outputFeatures = contentFeatures[2, :, :, :]

        # compute the feature reconstruction loss, weighting it
        # appropriately
        contentLoss = self.featureReconLoss(styleFeatures, outputFeatures)
        contentLoss = contentLoss * self.S["content_weight"]

        # initialize our style loss along with the value used to
        # weight each style layer (in proportion to the total number
        # of style layers
        styleLoss = K.variable(0.0)
        weight = 1.0 / len(self.S["style_layers"])

        # loop over the style layers
        for layer in self.S["style_layers"]:
            # grab the current style layer and use it to extract the
            # style features and output features from the *style
            # layer*
            styleOutput = layerMap[layer]
            styleFeatures = styleOutput[1, :, :, :]
            outputFeatures = styleOutput[2, :, :, :]

            # compute the style reconstruction loss as we go
            T = self.styleReconLoss(styleFeatures, outputFeatures)
            styleLoss = styleLoss + (weight * T)

        # finish computing the style loss, compute the total
        # variational loss, and then compute the total loss that
        # combines all three
        styleLoss = styleLoss * self.S["style_weight"]
        tvLoss = self.S["tv_weight"] * self.tvLoss(self.output)
        totalLoss = contentLoss + styleLoss + tvLoss

        # compute the gradients out of the output image with respect
        # to loss
        grads = K.gradients(totalLoss, self.output)
        outputs = [totalLoss]
        outputs = outputs + grads

        # the implementation of L-BFGS we will be using requires that
        # our loss and gradients be *two separate functions* so here
        # we create a Keras function that can compute both the loss
        # and gradients together and then return each separately
        # using two different class methods
        self.lossAndGrads = K.function([self.output], outputs)

    def preprocess(self, p):
        # load the input image (while resizing it to the desired
        # dimensions) and preprocess it
        image = load_img(p, target_size=self.dims)
        image = img_to_array(image)
        image = np.expand_dims(image, axis=0)
        image = preprocess_input(image)

        # return the preprocessed image
        return image

    def deprocess(self, image):
        # reshape the image, then reverse the zero-centering by
        # *adding* back in the mean values across the ImageNet
        # training set
        image = image.reshape((self.dims[0], self.dims[1], 3))
        image[:, :, 0] += 103.939
        image[:, :, 1] += 116.779
        image[:, :, 2] += 123.680

        # clip any values falling outside the range [0, 255] and
        # convert the image to an unsigned 8-bit integer
        image = np.clip(image, 0, 255).astype("uint8")

        # return the deprocessed image
        return image

    def gramMat(self, X):
        # the gram matrix is the dot product between the input
        # vectors and their respective transpose
        features = K.permute_dimensions(X, (2, 0, 1))
        features = K.batch_flatten(features)
        features = K.dot(features, K.transpose(features))

        # return the gram matrix
        return features

    def featureReconLoss(self, styleFeatures, outputFeatures):
        # the feature reconstruction loss is the squared error
        # between the style features and output output features
        return K.sum(K.square(outputFeatures - styleFeatures))

    def styleReconLoss(self, styleFeatures, outputFeatures):
        # compute the style reconstruction loss where A is the gram
        # matrix for the style image and G is the gram matrix for the
        # generated image
        A = self.gramMat(styleFeatures)
        G = self.gramMat(outputFeatures)

        # compute the scaling factor of the style loss, then finish
        # computing the style reconstruction loss
        scale = 1.0 / float((2 * 3 * self.dims[0] * self.dims[1]) ** 2)
        loss = scale * K.sum(K.square(G - A))

        # return the style reconstruction loss
        return loss

    def tvLoss(self, X):
        # the total variational loss encourages spatial smoothness in
        # the output page -- here we avoid border pixels to avoid
        # artifacts
        (h, w) = self.dims
        A = K.square(X[:, :h - 1, :w - 1, :] - X[:, 1:, :w - 1, :])
        B = K.square(X[:, :h - 1, :w - 1, :] - X[:, :h - 1, 1:, :])
        loss = K.sum(K.pow(A + B, 1.25))

        # return the total variational loss
        return loss

    def transfer(self, maxEvals=20):
        # generate a random noise image that will serve as a
        # placeholder array, slowly modified as we run L-BFGS to
        # apply style transfer
        X = np.random.uniform(0, 255, (1, self.dims[0], self.dims[1], 3)) - 128

        # start looping over the desired number of iterations
        for i in range(0, self.S["iterations"]):
            # run L-BFGS over the pixels in our generated image to
            # minimize the neural style loss
            print("[INFO] starting iteration {} of {}...".format(i + 1, self.S["iterations"]))
            (X, loss, _) = fmin_l_bfgs_b(self.loss, X.flatten(), fprime=self.grads, maxfun=maxEvals)
            print("[INFO] end of iteration {}, loss: {:.4e}".format(i + 1, loss))

            # deprocess the generated image and write it to disk
            image = self.deprocess(X.copy())
            p = os.path.sep.join([self.S["output_path"], "iter_{}.png".format(i)])
            cv2.imwrite(p, image)

    def loss(self, X):
        # extract the loss value
        X = X.reshape((1, self.dims[0], self.dims[1], 3))
        lossValue = self.lossAndGrads([X])[0]

        # return the loss
        return lossValue

    def grads(self, X):
        # compute the loss and gradients
        X = X.reshape((1, self.dims[0], self.dims[1], 3))
        output = self.lossAndGrads([X])

        # extract and return the gradient values
        return output[1].flatten().astype("float64")

Settings

In [4]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [5]:
# import the necessary packages
from keras.applications import VGG19

# initialize the settings dictionary
SETTINGS = {
	# initialize the path to the input (i.e., content) image,
	# style image, and path to the output directory
	"input_path": "drive/MyDrive/pyimagesearch/datasets/neural-style/jurassic_park.jpg",
	"style_path": "drive/MyDrive/pyimagesearch/datasets/neural-style/mcescher.jpg",
	"output_path": "drive/MyDrive/pyimagesearch/output/33-neural-style",

	# define the CNN to be used style transfer, along with the
	# set of content layer and style layers, respectively
	"net": VGG19,
	"content_layer": "block4_conv2",
	"style_layers": ["block1_conv1", "block2_conv1",
		"block3_conv1", "block4_conv1", "block5_conv1"],

	# store the content, style, and total variation weights,
	# respectively
	"content_weight": 1.0,
	"style_weight": 100.0,
	"tv_weight": 10.0,

	# number of iterations
	"iterations": 50,
}

# perform neural style transfer
ns = NeuralStyle(SETTINGS)
ns.transfer()

[INFO] loading network...
80142336/80134624 [==============================] - 0s 0us/step
[INFO] starting iteration 1 of 50...
[INFO] end of iteration 1, loss: 2.6831e+12
[INFO] starting iteration 2 of 50...
[INFO] end of iteration 2, loss: 1.4092e+12
[INFO] starting iteration 3 of 50...
[INFO] end of iteration 3, loss: 8.4431e+11
[INFO] starting iteration 4 of 50...
[INFO] end of iteration 4, loss: 6.0991e+11
[INFO] starting iteration 5 of 50...
[INFO] end of iteration 5, loss: 4.9530e+11
[INFO] starting iteration 6 of 50...
[INFO] end of iteration 6, loss: 4.3392e+11
[INFO] starting iteration 7 of 50...
[INFO] end of iteration 7, loss: 3.8988e+11
[INFO] starting iteration 8 of 50...
[INFO] end of iteration 8, loss: 3.6184e+11
[INFO] starting iteration 9 of 50...
[INFO] end of iteration 9, loss: 3.4381e+11
[INFO] starting iteration 10 of 50...
[INFO] end of iteration 10, loss: 3.2868e+11
[INFO] starting iteration 11 of 50...
[INFO] end of iteration 11, loss: 3.1867e+11
[INFO] startin